In [1]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp as mc



In [ ]:
pip install pandas xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 1.2 MB/s eta 0:00:00


In [ ]:
df_new = pd.read_excel('overalldata.xlsx', sheet_name='Sheet1')

exclude_models = [
    "codellama-70b-instruct@anyscale",
    "codellama-7b-instruct@together-ai",
    "command-r-plus@aws-bedrock",
    "deepseek-coder-33b-instruct@together-ai",
    "phi-3-medium-4k-instruct@deepinfra"
]

filtered_df = df_new[~df_new['model'].isin(exclude_models)]

control_group = filtered_df[filtered_df['model'] == 'original_text']
other_groups = filtered_df[filtered_df['model'] != 'original_text']

combined_df = pd.concat([control_group, other_groups])

model = ols('Q("Automated Readability Index") ~ C(model)', data=combined_df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

print(anova_table)

# Tukey's HSD test
comp = mc.pairwise_tukeyhsd(combined_df['Automated Readability Index'], combined_df['model'])
tukey_results = pd.DataFrame(data=comp.summary().data[1:], columns=comp.summary().data[0])

control_comparisons = tukey_results[tukey_results['group1'] == 'original_text']

# Sort by absolute mean difference to find the most similar models
control_comparisons_sorted = control_comparisons.sort_values(by='meandiff', key=abs)
print(control_comparisons_sorted.head())


              sum_sq     df         F    PR(>F)
C(model)  225.937579   28.0  2.768547  0.000012
Residual  798.599714  274.0       NaN       NaN
            group1                         group2  meandiff   p-adj   lower  \
405  original_text  qwen-2-72b-instruct@deepinfra    2.5371  0.0232  0.1439   

      upper  reject  
405  4.9304    True  


In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

df_new = pd.read_excel('overalldata.xlsx', sheet_name='Sheet1')

exclude_models = [
    "codellama-70b-instruct@anyscale",
    "codellama-7b-instruct@together-ai",
    "command-r-plus@aws-bedrock",
    "deepseek-coder-33b-instruct@together-ai",
    "phi-3-medium-4k-instruct@deepinfra",
    "nemotron-4-340b-instruct@deepinfra",
    "codellama-13b-instruct@together-ai"
]

filtered_df = df_new[~df_new['model'].isin(exclude_models)]

# summary table for the 'Flesch Reading Ease' grouped by 'model'
summary_table = filtered_df.groupby('model')['Automated Readability Index'].describe()

print(summary_table)

# Define and fit the ANOVA model
model = ols('Q("Automated Readability Index") ~ C(model)', data=filtered_df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)


                                        count       mean       std   min  \
model                                                                      
claude-3-haiku@anthropic                 15.0   9.020000  1.558937   6.5   
claude-3-opus@anthropic                  15.0   8.906667  1.864889   5.9   
claude-3-sonnet@anthropic                 1.0  12.600000       NaN  12.6   
claude-3.5-sonnet@anthropic              15.0   8.353333  1.380924   5.5   
gemini-1.5-flash@vertex-ai               15.0   8.653333  1.428719   6.5   
gemini-1.5-pro@vertex-ai                 15.0   8.206667  0.982320   6.8   
gemma-2-9b-it@fireworks-ai               15.0   8.906667  1.685428   6.0   
gemma-2b-it@together-ai                  14.0   9.335714  2.016321   6.2   
gemma-7b-it@anyscale                      1.0  10.100000       NaN  10.1   
gpt-3.5-turbo@openai                     15.0   9.693333  1.568196   6.9   
gpt-4-turbo@openai                       15.0   8.853333  1.417678   6.1   
gpt-4@openai

In [ ]:
import pandas as pd

anova_df = pd.DataFrame(anova_table)

output_file = 'anova_table_output_Automated Readability Index.xlsx'

anova_df.to_excel(output_file, index=True)

print(f"ANOVA table saved as {output_file}")

ANOVA table saved as anova_table_output_Automated Readability Index.xlsx


In [ ]:
summary_table = filtered_df.groupby('model')['Automated Readability Index'].describe()
anova_df = pd.DataFrame(anova_table)

output_file_path = 'summary_and_anova_output_Automated Readability Index.xlsx'

with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
    summary_table.to_excel(writer, sheet_name='Summary Statistics')
    anova_df.to_excel(writer, sheet_name='ANOVA Table')

print(f"Data has been saved to {output_file_path}")

Data has been saved to summary_and_anova_output_Automated Readability Index.xlsx
